In [0]:
!pip install vega
!pip install altair vega_datasets

     |████████████████████████████████| 1.2MB 2.8MB/s 


In [0]:
%cd /content/drive/My Drive/wine_categorization

/content/drive/My Drive/wine_categorization


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pickle
import seaborn as sns
sns.set_style("whitegrid")
import altair as alt
#alt.renderers.enable("notebook")
alt.renderers.enable('colab')
#alt.renderers.enable('jupyterlab/notebook')
from IPython.display import  display




In [0]:
df = pd.read_csv('/content/drive/My Drive/wine_categorization/data/train.csv')
df.shape

(82657, 12)

In [0]:
df.isna().sum()

user_name             19393
country                  35
review_title              0
review_description        0
designation           23647
points                    0
price                  5569
province                 35
region_1              12754
region_2              46708
winery                    0
variety                   0
dtype: int64

In [0]:
df_1 = df.loc[:5000]

In [0]:
bars = alt.Chart(df_1).mark_bar(size=50).encode(
    x=alt.X("variety"),
    y=alt.Y("count():Q", axis=alt.Axis(title='Number of entries')),
    tooltip=[alt.Tooltip('count()', title='Number of entries'), 'variety'],
    color='variety'

)

text = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='count()'
)

(bars + text).interactive().properties(
    height=500, 
    width=1500,
    title = "Number of entries in each category",
)

In [0]:
df_1['id'] = 1
df2 = pd.DataFrame(df_1.groupby('variety').count()['id']).reset_index()

bars = alt.Chart(df2).mark_bar(size=50).encode(
    x=alt.X('variety'),
    y=alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.0%', title='% of entries')),
    color='variety'
).transform_window(
    TotalArticles='sum(id)',
    frame=[None, None]
).transform_calculate(
    PercentOfTotal="datum.id / datum.TotalArticles"
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    #dx=5  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('PercentOfTotal:Q', format='.1%')
)

(bars + text).interactive().properties(
    height=300, 
    width=1500,
    title = "% of entries in each category",
)

alt.LayerChart(...)

In [0]:
df.columns

Index(['user_name', 'country', 'review_title', 'review_description',
       'designation', 'points', 'price', 'province', 'region_1', 'region_2',
       'winery', 'variety'],
      dtype='object')

In [0]:
df_1 = df.loc[df.points.notna(),['review_title', 'review_description', 'points', 'variety']]
display(df_1.head)

<bound method NDFrame.head of                                             review_title  ...                     variety
0      Andrew Peace 2007 Peace Family Vineyard Chardo...  ...                  Chardonnay
1      North by Northwest 2014 Red (Columbia Valley (...  ...                   Red Blend
2                      Renato Ratti 2007 Conca  (Barolo)  ...                    Nebbiolo
3      Domaine l'Ancienne Cure 2010 L'Abbaye White (B...  ...  Bordeaux-style White Blend
4      Château du Cèdre 2012 Le Cèdre Vintage Malbec ...  ...                      Malbec
...                                                  ...  ...                         ...
82652                   Montecillo 2007 Crianza  (Rioja)  ...                 Tempranillo
82653  Vivác Winery 2008 Single Vineyard Cabernet Sau...  ...          Cabernet Sauvignon
82654  Domaine de la Madone 2014 Nouveau  (Beaujolais...  ...                       Gamay
82655                  Château Beaumont 2011  Haut-Médoc  ...    Borde

In [0]:
with open('/content/drive/My Drive/wine_categorization/model/News_dataset.pickle', 'wb') as output:
    pickle.dump(df_1, output)

In [0]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [0]:
path_df = "/content/drive/My Drive/wine_categorization/model/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [0]:

df.loc[6]['review_description']

"Half Syrah and half Cabernet Sauvignon, with 40% new French oak, this wine draws you in with its notes of blackberries, milk chocolate and anise. It's broad and fleshy in feel, with abundant coffee and cherry flavors backed by ample acidity."

In [0]:
df['Description_Parsed_1'] = df['review_description'].str.replace('"', '')

In [0]:
df['Description_Parsed_2'] = df['Description_Parsed_1'].str.lower()

In [0]:
punctuation_signs = list("?:!.,;")
df['Description_Parsed_3'] = df['Description_Parsed_2']

for punct_sign in punctuation_signs:
    df['Description_Parsed_3'] = df['Description_Parsed_3'].str.replace(punct_sign, '')

In [0]:

df['Description_Parsed_4'] = df['Description_Parsed_3'].str.replace("'s", "")

In [0]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Description_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [0]:
df['Description_Parsed_5'] = lemmatized_text_list

In [0]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
stop_words[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [0]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [0]:
df['Description_Parsed_6'] = df['Description_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Description_Parsed_6'] = df['Description_Parsed_6'].str.replace(regex_stopword, '')

In [0]:
df.loc[5]['review_description']

"Strident aromas of licorice, herbs, berries and mint make it interesting, while the palate is light and fresh, with raspberry and red currant flavors. There's not a lot of heft, extract or depth to the wine, but it's likable and unhindered by excess oak and such. Enjoyable due to its unchallenging simplicity."

In [0]:
df.loc[5]['Description_Parsed_1']

"Strident aromas of licorice, herbs, berries and mint make it interesting, while the palate is light and fresh, with raspberry and red currant flavors. There's not a lot of heft, extract or depth to the wine, but it's likable and unhindered by excess oak and such. Enjoyable due to its unchallenging simplicity."

In [0]:
df.loc[5]['Description_Parsed_2']

"strident aromas of licorice, herbs, berries and mint make it interesting, while the palate is light and fresh, with raspberry and red currant flavors. there's not a lot of heft, extract or depth to the wine, but it's likable and unhindered by excess oak and such. enjoyable due to its unchallenging simplicity."

In [0]:
df.loc[5]['Description_Parsed_3']

"strident aromas of licorice herbs berries and mint make it interesting while the palate is light and fresh with raspberry and red currant flavors there's not a lot of heft extract or depth to the wine but it's likable and unhindered by excess oak and such enjoyable due to its unchallenging simplicity"

In [0]:
df.loc[5]['Description_Parsed_4']

'strident aromas of licorice herbs berries and mint make it interesting while the palate is light and fresh with raspberry and red currant flavors there not a lot of heft extract or depth to the wine but it likable and unhindered by excess oak and such enjoyable due to its unchallenging simplicity'

In [0]:
df.loc[5]['Description_Parsed_5']

'strident aromas of licorice herbs berry and mint make it interest while the palate be light and fresh with raspberry and red currant flavor there not a lot of heft extract or depth to the wine but it likable and unhindered by excess oak and such enjoyable due to its unchallenging simplicity'

In [0]:
df['Description_Parsed_7']=df['Description_Parsed_6'].str.replace("'","")

In [0]:
df.loc[6]['Description_Parsed_7']

'half syrah  half cabernet sauvignon  40% new french oak  wine draw     note  blackberry milk chocolate  anise  broad  fleshy  feel  abundant coffee  cherry flavor back  ample acidity'

In [0]:
df.head(1)

,review_title,review_description,points,variety,Description_Parsed_1,Description_Parsed_2,Description_Parsed_3,Description_Parsed_4,Description_Parsed_5,Description_Parsed_6,Description_Parsed_7
0,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",83,Chardonnay,"Classic Chardonnay aromas of apple, pear and h...","classic chardonnay aromas of apple, pear and h...",classic chardonnay aromas of apple pear and ha...,classic chardonnay aromas of apple pear and ha...,classic chardonnay aromas of apple pear and ha...,classic chardonnay aromas apple pear hay lea...,classic chardonnay aromas apple pear hay lea...


In [0]:
list_columns = ["review_title", "review_description", "Description_Parsed_7", "points", "variety"]
df = df[list_columns]

df = df.rename(columns={'Description_Parsed_7': 'Description_Parsed'})

In [0]:
df.head(1)

,review_title,review_description,Description_Parsed,points,variety
0,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",classic chardonnay aromas apple pear hay lea...,83,Chardonnay


In [0]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

In [0]:
variety_codes = {
    'Bordeaux-style Red Blend':0,
    'Bordeaux-style White Blend':1,
    'Cabernet Franc': 2,
    'Cabernet Sauvignon':3,
    'Champagne Blend':4,
    'Chardonnay':5,
    'Gamay':6,
    'Gewürztraminer':7,
    'Grüner Veltliner':8,
    'Malbec':9,
    'Merlot':10,
    'Nebbiolo':11,
    'Pinot Grigio':12,
    'Pinot Gris':13,
    'Pinot Noir':14,
    'Portuguese Red':15,
    'Portuguese White':16,
    'Red Blend':17,
    'Rhône-style Red Blend':18,
    'Riesling':19,
    'Rosé':20,
    'Sangiovese':21,
    'Sauvignon Blanc':22,
    'Sparkling Blend':23,
    'Syrah':24,
    'Tempranillo':25,
    'White Blend':26,
    'Zinfandel':27
}

In [0]:
# Category mapping
df['Variety_Code'] = df['variety']
df = df.replace({'Variety_Code':variety_codes})

In [0]:
df.head(15)

,review_title,review_description,Description_Parsed,points,variety,Variety_Code
0,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",classic chardonnay aromas apple pear hay lea...,83,Chardonnay,5
1,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,wine near equal part syrah merlot balance...,89,Red Blend,17
2,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,barolo conca open inky dark concentration so...,94,Nebbiolo,11
3,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,impressive small addition sauvignon gris ...,87,Bordeaux-style White Blend,1
4,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",ripe sweet wine rich full dry fresh fruit...,88,Malbec,9
5,Andean 2005 Finca La Escondida Reserva Caberne...,"Strident aromas of licorice, herbs, berries an...",strident aromas licorice herbs berry mint ma...,85,Cabernet Sauvignon,3
6,Gordon Estate 2012 Tradition Red (Columbia Val...,"Half Syrah and half Cabernet Sauvignon, with 4...",half syrah half cabernet sauvignon 40% new f...,91,Red Blend,17
7,Rock Wall 2014 Monte Rosso Reserve Zinfandel (...,"High-toned red fruit stars in this wine, a nic...",high-toned red fruit star wine nice show ...,90,Zinfandel,27
8,Elizabeth Chambers 2014 Falcon Glen Vineyard P...,"Displaying typical McMinnville AVA subtlety, t...",display typical mcminnville ava subtlety fram...,91,Pinot Noir,14
9,Mannucci Droandi 2009 Ceppeto Riserva (Chiant...,This blend of Sangiovese and 10% Merlot opens ...,blend sangiovese 10% merlot open aromas f...,86,Red Blend,17


In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['Description_Parsed'], 
                                                    df['Variety_Code'], 
                                                    test_size=0.2, 
                                                    random_state=8)

In [0]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [0]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)
print(labels_train.shape)
features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)
print(labels_test.shape)

(66125, 300)
(66125,)
(16532, 300)
(16532,)


In [0]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(variety_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Bordeaux-style Red Blend' category:
  . Most correlated unigrams:
. structure
. cabernet
. wood
. merlot
. franc
  . Most correlated bigrams:
. cabernet franc
. black currant

# 'Bordeaux-style White Blend' category:
  . Most correlated unigrams:
. crisp
. apricot
. aftertaste
. character
. blanc
  . Most correlated bigrams:
. black cherry
. ready drink

# 'Cabernet Franc' category:
  . Most correlated unigrams:
. red
. herb
. cherry
. cabernet
. franc
  . Most correlated bigrams:
. cherry flavor
. cabernet franc

# 'Cabernet Sauvignon' category:
  . Most correlated unigrams:
. currant
. cedar
. blackberry
. cabernet
. cassis
  . Most correlated bigrams:
. pinot noir
. black currant

# 'Champagne Blend' category:
  . Most correlated unigrams:
. bottle
. mature
. crisp
. ready
. aftertaste
  . Most correlated bigrams:
. pinot noir
. ready drink

# 'Chardonnay' category:
  . Most correlated unigrams:
. tropical
. pineapple
. apple
. butter
. chardonnay
  . Most correlated bigrams:
. b

In [0]:

bigrams

['fruit flavor',
 'red cherry',
 'medium bodied',
 'bake spice',
 'white pepper',
 'red fruit',
 'red berry',
 'berry fruit',
 'cherry flavor',
 'black cherry',
 'firm tannins',
 'cherry fruit',
 'black currant',
 'black fruit',
 'cabernet franc',
 'pinot noir',
 'drink 2018',
 'palate offer',
 'ready drink',
 'cabernet sauvignon',
 'full bodied',
 'black pepper']

In [0]:
%cd /content/drive/My Drive/wine_categorization/model/

/content/drive/My Drive/wine_categorization/model


In [0]:
# X_train
with open('Pickle/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickle/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickle/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickle/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickle/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickle/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickle/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickle/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickle/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickle/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [0]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Dataframe
path_df = "/content/drive/My Drive/wine_categorization/model/Pickle/df.pickle"
with open(path_df, 'rb') as data:
    df = pickle.load(data)

# features_train
path_features_train = "/content/drive/My Drive/wine_categorization/model/Pickle/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "/content/drive/My Drive/wine_categorization/model/Pickle/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "/content/drive/My Drive/wine_categorization/model/Pickle/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "/content/drive/My Drive/wine_categorization/model/Pickle/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)

In [0]:
print(features_train.shape)
print(labels_train.shape)
print(features_test.shape)

print(labels_test.shape)

(66125, 300)
(66125,)
(16532, 300)
(16532,)


In [0]:
svc_0 =svm.SVC(random_state=8)

print('Parameters currently in use:\n')
pprint(svc_0.get_params())

Parameters currently in use:

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 8,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


In [0]:
# C
C = [.0001, .001, .01]

# gamma
gamma = [.0001, .001, .01, .1, 1, 10, 100]

# degree
degree = [1, 2, 3, 4, 5]

# kernel
kernel = ['linear', 'rbf', 'poly']

# probability
probability = [True]

# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
              'probability': probability
             }

pprint(random_grid)

{'C': [0.0001, 0.001, 0.01],
 'degree': [1, 2, 3, 4, 5],
 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
 'kernel': ['linear', 'rbf', 'poly'],
 'probability': [True]}


In [0]:
# C
C = [.01]

# gamma
gamma = [10]

# degree
degree = [4, 5]

# kernel
kernel = ['poly']

# probability
probability = [True]

# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
              'probability': probability
             }

pprint(random_grid)

{'C': [0.01],
 'degree': [4, 5],
 'gamma': [10],
 'kernel': ['poly'],
 'probability': [True]}


In [0]:
# First create the base model to tune
svc = svm.SVC(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=svc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(features_train, labels_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits


In [0]:
print("The best hyperparameters from Grid Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

In [0]:
best_svc = grid_search.best_estimator_
best_svc

In [0]:
with open('/content/drive/My Drive/wine_categorization/model/best_svc.pickle', 'wb') as output:
    pickle.dump(best_svc, output)